<a href="https://colab.research.google.com/github/Otitoaleke/data_incubator_challenge/blob/master/TDI_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Number 2

## Import libraries

In [0]:
import os
import numpy as np
import pandas as pd
import math
import scipy.stats

## Import the dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/PartD_Prescriber_PUF_NPI_17.txt", sep='\t')
data_list = df.values.tolist()

In [0]:
def extract(index, data_list):
    column = []    
    for row in data_list[0:]:
        #print(row)
        value = row[index]
        column.append(value)    
    return column

In [5]:
bene_count = extract(21, data_list)

print(type(bene_count))
counted_bene_count = []

<class 'list'>


In 2017, what was the average number of beneficiaries per provider? Due to the suppression of data for those with few beneficiaries, we can only include those with more than 10 beneficiaries. 

In [6]:
def average_bene_count():
    for item in bene_count:
        entry = float(item)
        if math.isnan(entry) == False:
            counted_bene_count.append(entry)
    #print(counted_bene_count)
    average_no_of_bene = sum(counted_bene_count)/len(counted_bene_count)
    return(average_no_of_bene)

print(average_bene_count())

158.3494585173676


In [0]:
specialty_collate = {}

for row in data_list:
    specialty = row[14]
    if math.isnan(row[30]) == False:
        if specialty in specialty_collate:
            specialty_collate[specialty]= [specialty_collate[specialty][0] + row[17], specialty_collate[specialty][1]+ row[30]]
        else:
            specialty_collate[specialty]= [ row[17], row[30]]
#print(specialty_collate)



Work out for each Specialty the fraction of drug claims that are for brand-name drugs. Include only providers for whom the relevant information has not been suppressed, and consider only specialties with at least 1000 total claims. What is the standard deviation of these fractions? 

In [8]:
fractions = []
for specialty in specialty_collate:
    fractions.append(specialty_collate[specialty][1]/specialty_collate[specialty][0])

#print(fractions)
standard_deviation = np.std(fractions)
print(standard_deviation)


0.13218193916228735


Let's find which states have surprisingly high supply of opioids, conditioned on specialty. Work out the average length of an opioid prescription for each provider. For each (state, specialty) pair with at least 100 providers, calculate the average of this value across all providers. Then find the ratio of this value to an equivalent quantity calculated from providers in each specialty across all states. What is the largest such ratio? 

In [0]:
#average length of opioid prescription = Number of day's ssupply of all opioid drugs ( col 53) / total claims of opioid drugs (col 51)

avg_len_op_pres = []
for row in data_list:
    if float(row[50]) != 0 and math.isnan(row[52]) == False: 
        avg_len_op_pres.append([row[0], float(row[52])/ float(row[50]), row[12], row[14]])

#print(avg_len_op_pres[:20])



In [0]:
specialty_group = {}
for item in avg_len_op_pres:
    specialty = item[3]

    if specialty in specialty_group:
        specialty_group[specialty].append(item[1])
        
    else:
        specialty_group[specialty] = [(item[1])]
       


In [0]:
#mean within specialty groups
specialty_averages = {}

for specialty in specialty_group:
    specialty_averages[specialty] = np.mean(specialty_group[specialty])

#print(specialty_averages)

In [0]:
specialty_avg_in_states = {}
for item in avg_len_op_pres:
    specialty = item[3]
    state = item[2]

    if state in specialty_avg_in_states:
        if specialty in specialty_avg_in_states[state]:
           specialty_avg_in_states[state][specialty].append(item[1])
        else:
           specialty_avg_in_states[state][specialty] = [(item[1])]

    else:
        specialty_avg_in_states[state] = {}

        if specialty in specialty_avg_in_states[state]:
           specialty_avg_in_states[state][specialty].append(item[1])
        else:
           specialty_avg_in_states[state][specialty] = [(item[1])]


In [13]:
state_specialty_avg_dict = {}
state_specialty_ratio_list = []
for state in specialty_avg_in_states:
    for specialty in specialty_avg_in_states[state]:
        if len(specialty_avg_in_states[state][specialty]) >=100:
            mean_vals = np.mean(specialty_avg_in_states[state][specialty])
            state_specialty_avg_dict[state]= {specialty: mean_vals}
            state_specialty_ratio_list.append(mean_vals/specialty_averages[specialty])

#print(state_specialty_avg_dict)
#print(state_specialty_ratio_list)

largest_ratio = np.max(state_specialty_ratio_list)
print(largest_ratio)

1.4851731533528136


For each provider, estimate the length of the average prescription from the total_day_supply and total_claim_count. What is the median, in days, of the distribution of this value across all providers? 

In [14]:
prescription_length = []
for row in data_list:
    prescription_length.append(row[20]/row[17])

prescription_median = np.median(prescription_length)
print(prescription_median)


29.7125748502994


Find the ratio of beneficiaries with opioid prescriptions to beneficiaries with antibiotics prescriptions in each state. Assume that each beneficiary attends only a single provider. What is the difference between the largest and smallest ratios? 

In [15]:
state_collate = {}

for row in data_list:
    state = row[12]
    if state in state_collate:
        #state_collate[state]=[]
        if math.isnan(row[53]) == False:
            state_collate[state][0] += row[53]
        if math.isnan(row[62]) == False : 
            state_collate[state][1] += row[62]
    else:
        state_collate[state]=[]
        if math.isnan(row[53]) == False:
            state_collate[state].append(row[53])
        else:
            state_collate[state].append(0)
        if math.isnan(row[62]) == False : 
            state_collate[state].append(row[62])
        else:
            state_collate[state].append(0)
#print(state_collate)
        
ratios = []
for state in state_collate:
    ratios.append(state_collate[state][0]/state_collate[state][1])

ratio_range = np.max(ratios) - np.min(ratios)

print(ratio_range)

#standard_deviation = np.std(fractions)
#print(standard_deviation)

0.6331097230267027


For each provider where the relevant columns are not suppressed, work out the fraction of claims for beneficiaries age 65 and older, as well as the fraction of claims for beneficiaries with a low-income subsidy. What is the Pearson correlation coefficient between these values? 

In [16]:
bene_count_65 = extract(23, data_list)
bene_count_LI = extract(45, data_list)

counted_bene_count_65 = []
counted_bene_count_LI = []

for item in bene_count_65:
    entry = float(item)
    if math.isnan(entry) == False:
        counted_bene_count_65.append(entry)
sum_65 = sum(counted_bene_count_65)
mean_65 = np.mean(counted_bene_count_65)

for item in bene_count_LI:
    entry = float(item)
    if math.isnan(entry) == False:
        counted_bene_count_LI.append(entry)
sum_LI = sum(counted_bene_count_LI)
mean_LI = np.mean(counted_bene_count_LI)

new_bene_count_65 = []
new_bene_count_LI = []

for item in bene_count_65:
    entry = float(item)
    if math.isnan(entry) == False:
        new_bene_count_65.append(entry)
    else:
        new_bene_count_65.append(mean_65)

for item in bene_count_LI:
    entry = float(item)
    if math.isnan(entry) == False:
        new_bene_count_LI.append(entry)
    else:
        new_bene_count_LI.append(mean_LI)
    
frac_65 = (1./sum(new_bene_count_65)) * np.array(new_bene_count_65)
frac_LI = (1./sum(new_bene_count_LI)) * np.array((new_bene_count_LI))

#print(len(frac_65), len(frac_LI))
#print(frac_65[:20])
#print(frac_LI[:20])

pearson_corr = scipy.stats.pearsonr(frac_65, frac_LI) 

print(pearson_corr)


(0.7968910027092642, 0.0)


For each provider for whom the information is not suppressed, figure out the average cost per day of prescriptions in both 2016 and 2017. Use this to estimate the inflation rate for daily prescription costs per provider. What is the average inflation rate across all providers? 

In [0]:
df_16 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/PartD_Prescriber_PUF_NPI_16.txt", sep='\t')
data_list_16 = df_16.values.tolist()

In [18]:
cost_per_day_17 = []
for row in data_list:
    total_day = row[20]
    total_cost = row[19]
    cost_per_day_17.append(total_cost/total_day)
mean_cost_per_day_17 = np.mean(cost_per_day_17)
print(mean_cost_per_day_17)

4.200463873282938


In [19]:
cost_per_day_16 = []
for row in data_list_16:
    total_day = row[20]
    total_cost = row[19]
    cost_per_day_16.append(total_cost/total_day)
mean_cost_per_day_16 = np.mean(cost_per_day_16)
print(mean_cost_per_day_16)

4.160214951442098


In [20]:
inflation_rate = 100*(mean_cost_per_day_17 - mean_cost_per_day_16)/mean_cost_per_day_16

print("Average inflation rate across all providers is: %.2f" %(inflation_rate))

Average inflation rate across all providers is: 0.97


Consider all providers with a defined specialty in both years. Find the fraction of providers who left each specialty between 2016 and 2017. What is the largest such fraction, when considering specialties with at least 1000 proviers in 2016? Note that some specialties have a fraction of 1 due to specialty name changes between 2016 and 2017; disregard these specialties in calculating your answer. 

In [0]:
specialty_numbers_16 = {}

for row in data_list_16:
    specialty = row[14]
    if specialty in specialty_numbers_16:
        specialty_numbers_16[specialty] += 1
    else:
        specialty_numbers_16[specialty] = 1

#print(specialty_numbers_16)

In [0]:
specialty_numbers_17 = {}

for row in data_list:
    specialty = row[14]
    if specialty in specialty_numbers_17:
        specialty_numbers_17[specialty] += 1
    else:
        specialty_numbers_17[specialty] = 1

#print(specialty_numbers_17)

In [23]:
specialty_changes = {}
specialty_changes_list = []
specialty_changes_frac_list = []
for specialty in specialty_numbers_16:
    if specialty_numbers_16[specialty] >= 1000:
        if specialty in specialty_numbers_17:
            specialty_changes[specialty] = specialty_numbers_17[specialty]-specialty_numbers_16[specialty]
            specialty_changes_list.append(np.abs(specialty_changes[specialty]))
            specialty_changes_frac_list.append(np.abs(specialty_changes[specialty])/specialty_numbers_16[specialty])
        else:
            specialty_changes[specialty] = specialty_numbers_16[specialty]
            specialty_changes_list.append(np.abs(specialty_changes[specialty]))
            specialty_changes_frac_list.append(np.abs(specialty_changes[specialty])/specialty_numbers_16[specialty])

mod_specialty_changes_frac_list = [i for i in specialty_changes_frac_list if i!=1.0]
#print(mod_specialty_changes_frac_list)
largest_frac = np.max(mod_specialty_changes_frac_list)
print ("The largest fractional change is %0.3f" %(largest_frac))
#print(specialty_changes)
#print(specialty_changes_list)
#print(specialty_changes_frac_list)


The largest fractional change is 0.195


In [0]:
if state in state_collate:
        #state_collate[state]=[]
        if math.isnan(row[53]) == False:
            state_collate[state][0] += row[53]
        if math.isnan(row[62]) == False : 
            state_collate[state][1] += row[62]

# Number 3

In [0]:
def gen_samples(N):
  
  data = np.random.choice(np.arange(1, N+1), size = N, replace = False)
  samples = []
  for idx in range(N):
    if idx == 0:
      samples.append(data[idx])
    else:
      samples.append(abs(data[idx] - data[idx -1]))

  #print(data)
  #print(samples) 
  return samples

In [0]:
def gen_samples_v2(N, num_of_samples):

  samples = []
  for i in range(num_of_samples):
    samples.append(sum(gen_samples(N)))
  return np.array(samples)

For  N = 20

In [27]:
samples_ = gen_samples_v2(N = 20, num_of_samples = 1000000)
print("Mean is %.4f" %(np.mean(samples_)))
print("Standard Deviation is %.4f" %(np.std(samples_)))
print("Probability that total payment is greater than or eqaul to 160  is %.5f" %(np.mean(samples_ >= 160)))

Mean is 143.4805
Standard Deviation is 18.4475
Probability that total payment is greater than or eqaul to 160  is 0.19744


In [28]:
np.mean(samples_ >= 160)

0.197438

For N = 10

In [29]:
samples_10 = gen_samples_v2(N = 10, num_of_samples = 1000000)
print("Mean is %.4f" %(np.mean(samples_10)))
print("Standard Deviation is %.4f" %(np.std(samples_10)))
print("Probability that total payment is greater than or eqaul to 45 is %.5f" %(np.mean(samples_10 >= 45)))

Mean is 38.4999
Standard Deviation is 6.3665
Probability that total payment is greater than or eqaul to 45 is 0.18226
